In [1]:
# %env CUDA_DEVICE_ORDER=PCI_BUS_ID
# %env CUDA_VISIBLE_DEVICES="0"

In [2]:
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn import preprocessing
from datasets import Dataset
import evaluate
import numpy as np

2024-09-28 22:00:42.395479: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-28 22:00:43.097780: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
data_path = "retriever/data/train_val_test.json"

In [4]:
with open(data_path, 'r') as file:
    data = json.load(file)

In [5]:
train_df = pd.DataFrame(data['folds'][0]['train'])
val_df = pd.DataFrame(data['folds'][0]['val'])

In [6]:
train_df = train_df[train_df.classifier_1 != "ОТСУТСТВУЕТ"]
val_df = val_df[val_df.classifier_1 != "ОТСУТСТВУЕТ"]

In [7]:
le = preprocessing.LabelEncoder()
train_df['labels'] = le.fit_transform(train_df.classifier_2.values)
val_df['labels'] = le.fit_transform(val_df.classifier_2.values)

In [8]:
label2id = dict(zip(train_df.classifier_2, train_df.labels))
id2label = dict(zip(train_df.labels, train_df.classifier_2))

In [9]:
id2label

{34: 'Удаление аккаунта',
 14: 'Отклонение/блокировка видео',
 15: 'Отключение/подключение монетизации',
 6: 'Загрузка видео',
 11: 'Навигация',
 2: 'Верификация',
 23: 'Приложение',
 36: 'Управление трансляцией',
 20: 'Плеер',
 24: 'Продвижение канала',
 35: 'Управление плеером',
 17: 'Персонализация',
 10: 'Монетизация',
 28: 'Смена категории/возрастные ограничения',
 26: 'Регистрация/Авторизация',
 4: 'Встраивание видео',
 16: 'Перенос видео с Youtube',
 3: 'Воспроизведение видео',
 12: 'Нарушение авторских прав',
 5: 'Долгая модерация',
 33: 'Трансляция',
 29: 'Статистика по монетизации',
 32: 'Текстовый поиск',
 13: 'Недоступность видео',
 1: 'Блокировка канала',
 8: 'История поиска',
 30: 'Студия RUTUBE',
 27: 'Система рекомендаций',
 25: 'Просмотр трансляции',
 7: 'Запрещенный контент',
 19: 'Платный контент',
 21: 'Подключение/отключение донатов',
 0: 'Аналитика',
 22: 'Подключение/отключение рекламы',
 9: 'Комментарии',
 18: 'Персонализация 0',
 31: 'ТВ-эфиры',
 37: 'Чат/Комме

In [10]:
least_classes = train_df['classifier_2'].value_counts().tail(21).to_dict()

In [11]:
train_df[train_df['classifier_2'] == 'Плеер'][:100-88]

,question,response,classifier_1,classifier_2,is_knowledge_base,labels
143,В записи трансляции появятся субтитры?,"Сейчас такой функции нет, но мы зафиксировали ...",ПРЕДЛОЖЕНИЯ,Плеер,0,20
144,"Привет, а будут ли субтитры в записи трансляции?","Сейчас такой функции нет, но мы зафиксировали ...",ПРЕДЛОЖЕНИЯ,Плеер,0,20
145,"Добрый день, появятся ли субтитры в записи тра...","Сейчас такой функции нет, но мы зафиксировали ...",ПРЕДЛОЖЕНИЯ,Плеер,0,20
146,Будут субтитры на записи трансляции?,"Сейчас такой функции нет, но мы зафиксировали ...",ПРЕДЛОЖЕНИЯ,Плеер,0,20
147,Когда добавите субтитры в записи трансляций? Э...,"Сейчас такой функции нет, но мы зафиксировали ...",ПРЕДЛОЖЕНИЯ,Плеер,0,20
148,А субтитры в записи трансляции будут?,"Сейчас такой функции нет, но мы зафиксировали ...",ПРЕДЛОЖЕНИЯ,Плеер,0,20
149,"Скажите, планируются ли субтитры для записей т...","Сейчас такой функции нет, но мы зафиксировали ...",ПРЕДЛОЖЕНИЯ,Плеер,0,20
150,Появятся ли субтитры в записи трансляции?,"Сейчас такой функции нет, но мы зафиксировали ...",ПРЕДЛОЖЕНИЯ,Плеер,0,20
151,Есть ли возможность добавить субтитры к записи...,"Сейчас такой функции нет, но мы зафиксировали ...",ПРЕДЛОЖЕНИЯ,Плеер,0,20
152,Субтитры будут в записи трансляций?,"Сейчас такой функции нет, но мы зафиксировали ...",ПРЕДЛОЖЕНИЯ,Плеер,0,20


In [12]:
for k,v in least_classes.items():
    samples = train_df[train_df['classifier_2'] == k]
    if v == 55:
        new = pd.concat([samples, samples])
    elif v > 55:
        new = pd.concat([samples, samples[:110-len(samples)]])
    elif v < 55:
        new = pd.concat([samples, samples])
        while len(new) < 110:
            new = pd.concat([new, samples])
        else:
            new = pd.concat([new, samples[:110-v]])
    train_df = train_df[train_df['classifier_2'] != k]
    train_df = pd.concat([train_df, new])

In [13]:
train_df['classifier_2'].value_counts()

classifier_2
Персонализация                            1212
Управление трансляцией                     949
Загрузка видео                             898
Отключение/подключение монетизации         690
Регистрация/Авторизация                    561
Отклонение/блокировка видео                503
Студия RUTUBE                              451
Приложение                                 286
Верификация                                275
Встраивание видео                          226
Монетизация                                198
Трансляция                                 176
Платный контент                            176
История поиска                             165
Запрещенный контент                        165
Воспроизведение видео                      132
Просмотр трансляции                        132
Персонализация 0                           132
Комментарии                                132
Подключение/отключение рекламы             132
Чат/Комментарии                            132


In [14]:
train_ds = Dataset.from_pandas(train_df, split="train")
val_ds = Dataset.from_pandas(val_df, split="val")

In [15]:
train_ds =  train_ds.remove_columns(["response", "classifier_1", "classifier_2", "is_knowledge_base", "__index_level_0__"])
val_ds =  val_ds.remove_columns(["response", "classifier_1", "classifier_2", "is_knowledge_base", "__index_level_0__"])

In [16]:
train_ds =  train_ds.rename_column("question", "text")
val_ds =  val_ds.rename_column("question", "text")

In [17]:
train_ds[0]

{'text': 'ролики из инета защищены авторским правом?', 'labels': 14}

In [18]:
tokenizer = AutoTokenizer.from_pretrained("deepvk/deberta-v1-base")

/home/alex/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
def tokenize(batch):
    return tokenizer(batch["text"], padding='max_length', truncation=True, max_length=128, return_tensors="pt")

In [20]:
tokenized_train = train_ds.map(tokenize, batched=True)
tokenized_val = val_ds.map(tokenize, batched=True)

Map:   0%|          | 0/9725 [00:00<?, ? examples/s]

Map:   0%|          | 0/168 [00:00<?, ? examples/s]

In [21]:
tokenized_train = tokenized_train.shuffle(seed=0)
tokenized_val = tokenized_val.shuffle(seed=0)

In [22]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [23]:
metric = evaluate.load("f1")
metric1 = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)

    f1 = metric.compute(predictions=predictions, references=labels,
                         average="macro")["f1"]
    f1_w = metric.compute(predictions=predictions, references=labels,
                         average="weighted")["f1"]
    accuracy = metric1.compute(predictions=predictions, references=labels)[
        "accuracy"]
    
    return {"f1": f1,
            "f1_weighted": f1_w,
            "accuracy": accuracy}

In [24]:
model = AutoModelForSequenceClassification.from_pretrained(
    "deepvk/deberta-v1-base", num_labels=len(id2label), label2id=label2id, id2label=id2label
)

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at deepvk/deberta-v1-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="classifier_logs",
    learning_rate=2e-6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=6,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
    max_grad_norm=1.0,
    weight_decay=0.00001,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    logging_steps=100,
    warmup_steps=300,
    report_to="tensorboard",
    seed=42,
    use_cpu=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/home/alex/.local/lib/python3.8/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
trainer.train()

/home/alex/.local/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/alex/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,F1,F1 Weighted,Accuracy
1,3.274600,3.829794,0.103209,0.115801,0.148810
2,0.723700,5.186765,0.161074,0.191385,0.190476
3,0.302000,6.215043,0.171865,0.194149,0.202381
4,0.108900,6.644105,0.176844,0.209823,0.214286
5,0.049200,7.377625,0.153418,0.211968,0.214286
6,0.014500,7.548243,0.155661,0.210585,0.220238


/home/alex/.local/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/alex/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/alex/.local/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/alex/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to 

TrainOutput(global_step=912, training_loss=0.7069190523685202, metrics={'train_runtime': 393.0604, 'train_samples_per_second': 148.45, 'train_steps_per_second': 2.32, 'total_flos': 3838960122854400.0, 'train_loss': 0.7069190523685202, 'epoch': 6.0})